plan

1. check differences between bc, gail, airl in ppo and trpo.

2. for gail, check different network structure

# dependencies
connect to cpu if want to use behaviour cloning

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 28.6 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py

# GAIL-PPO

In [4]:
import gymnasium as gym
import imitation

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
SEED = 42

env = make_vec_env(
    "seals:seals/HalfCheetah-v1", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/HalfCheetah-v1",
    venv=env,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-seals-HalfCheetah-v1.zip:   0%|          | 0.00/168k [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
from imitation.data import rollout

rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=1000000, min_episodes=4), # config: https://huggingface.co/HumanCompatibleAI/ppo-seals-HalfCheetah-v1
    rng=np.random.default_rng(SEED), # 随机数？ The random state to use for sampling trajectories.
)

In [17]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=100,
    seed=SEED,
)

reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)

gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True
)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [18]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 20, return_episode_rewards=True
)

In [19]:
gail_trainer.train(300000)

round:   0%|          | 0/18 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | -370     |
|    gen/time/fps             | 2691     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.61     |
|    disc/disc_acc_expert             | 0.356    |
|    disc/disc_acc_gen                | 0.864    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.685    |
|    disc/disc_proportion_expert_pred | 0.246    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   6%|▌         | 1/18 [01:54<32:26, 114.50s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | -377       |
|    gen/rollout/ep_rew_wrapped_mean | 756        |
|    gen/time/fps                    | 2555       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 32768      |
|    gen/train/approx_kl             | 0.05130582 |
|    gen/train/clip_fraction         | 0.385      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -8.52      |
|    gen/train/explained_variance    | -0.626     |
|    gen/train/learning_rate         | 0.0004     |
|    gen/train/loss                  | 0.508      |
|    gen/train/n_updates             | 100        |
|    gen/train/policy_gradient_loss  | -0.0484    |
|    gen/train/std                   | 0.997      |
|    gen/tra

round:  11%|█         | 2/18 [03:49<30:34, 114.68s/it]

...

round:  89%|████████▉ | 16/18 [30:50<03:51, 115.95s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | -298       |
|    gen/rollout/ep_rew_wrapped_mean | 200        |
|    gen/time/fps                    | 2584       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 278528     |
|    gen/train/approx_kl             | 0.12011823 |
|    gen/train/clip_fraction         | 0.56       |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -7.4       |
|    gen/train/explained_variance    | 0.841      |
|    gen/train/learning_rate         | 0.0004     |
|    gen/train/loss                  | -0.0616    |
|    gen/train/n_updates             | 1600       |
|    gen/train/policy_gradient_loss  | -0.0644    |
|    gen/train/std                   | 0.828      |
|    gen/tra

round:  94%|█████████▍| 17/18 [32:45<01:55, 115.84s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | -282       |
|    gen/rollout/ep_rew_wrapped_mean | 161        |
|    gen/time/fps                    | 2502       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 294912     |
|    gen/train/approx_kl             | 0.13630891 |
|    gen/train/clip_fraction         | 0.583      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -7.32      |
|    gen/train/explained_variance    | 0.828      |
|    gen/train/learning_rate         | 0.0004     |
|    gen/train/loss                  | -0.0245    |
|    gen/train/n_updates             | 1700       |
|    gen/train/policy_gradient_loss  | -0.0663    |
|    gen/train/std                   | 0.818      |
|    gen/tra

round: 100%|██████████| 18/18 [34:41<00:00, 115.62s/it]


In [20]:
env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 20, return_episode_rewards=True
)

In [21]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -0.30919664999999996 +/- 0.5729603050948011
Rewards after training: 94.83383855 +/- 71.99733160835072


In [15]:
expert_reward, _ = evaluate_policy(
    expert, env, 8, return_episode_rewards=True
)

In [16]:
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1680.73820975 +/- 27.078724068659696


Cartpole

Rewards before training: 102.6 +/- 24.11514047232568
Rewards after training: 347.56 +/- 142.09801687567634

HaleCheetah

Rewards before training: -0.29786967999999997 +/- 0.6720326541318641
Rewards after training: 867.3461561099998 +/- 478.52644840867737
expert平均1669，看看是步数的问题还是别的参数。参考GAIL里面expert的结果，4000-5000分最终，运行300000到1000分，600000到1500，,2000000到2500

# GAIL-TRPO

In [ ]:
import gymnasium as gym
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

In [ ]:
SEED=42
env_TRPO = make_vec_env(
    "CartPole-v1",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)

In [ ]:
# seal cartpole v0: 别用它那个，要么从huggingface上下个看看，现在这个有ValueError: Observation spaces do not match
# 现在换了 cartpole v1
expert_TRPO = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="CartPole-v1",
    venv=env_TRPO,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-CartPole-v1.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from imitation.data import rollout

rollouts_TRPO = rollout.rollout(
    expert_TRPO,
    env_TRPO,
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=np.random.default_rng(SEED), # 随机数？ The random state to use for sampling trajectories.
)

In [ ]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO, RecurrentPPO

learner_TRPO = TRPO("MlpPolicy", env_TRPO, verbose=1) # RecurrentPPO("MlpLstmPolicy", env_TRPO, verbose=1)

reward_net = BasicRewardNet(
    observation_space=env_TRPO.observation_space,
    action_space=env_TRPO.action_space,
    normalize_input_layer=RunningNorm,
)

gail_trainer_TRPO = GAIL(
    demonstrations=rollouts_TRPO,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env_TRPO,
    gen_algo=learner_TRPO,
    reward_net=reward_net,
    allow_variable_horizon=True
)

Using cpu device
Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
env_TRPO.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_TRPO, env_TRPO, 100, return_episode_rewards=True
)

In [ ]:
gail_trainer_TRPO.train(200_000)

round:   0%|          | 0/12 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 23.8     |
|    gen/rollout/ep_rew_mean  | 23.8     |
|    gen/time/fps             | 3700     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 4        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.621    |
|    disc/disc_acc_expert             | 0.993    |
|    disc/disc_acc_gen                | 0.248    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.687    |
|    disc/disc_proportion_expert_pred | 0.873    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   8%|▊         | 1/12 [00:08<01:29,  8.18s/it]

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 29.2     |
|    gen/rollout/ep_rew_mean          | 29.2     |
|    gen/rollout/ep_rew_wrapped_mean  | 16.6     |
|    gen/time/fps                     | 2964     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 5        |
|    gen/time/total_timesteps         | 32768    |
|    gen/train/explained_variance     | -0.0111  |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.0084   |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 1        |
|    gen/train/policy_objective       | 0.0204   |
|    gen/train/value_loss             | 5.23     |
--------------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc             

round:  17%|█▋        | 2/12 [00:17<01:28,  8.81s/it]

...

round:  83%|████████▎ | 10/12 [01:29<00:18,  9.33s/it]

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 384      |
|    gen/rollout/ep_rew_mean          | 384      |
|    gen/rollout/ep_rew_wrapped_mean  | 197      |
|    gen/time/fps                     | 4033     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 4        |
|    gen/time/total_timesteps         | 180224   |
|    gen/train/explained_variance     | 0.949    |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.00791  |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 10       |
|    gen/train/policy_objective       | 0.0187   |
|    gen/train/value_loss             | 4.72     |
--------------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc             

round:  92%|█████████▏| 11/12 [01:38<00:09,  9.09s/it]

--------------------------------------------------
| raw/                                |          |
|    gen/rollout/ep_len_mean          | 445      |
|    gen/rollout/ep_rew_mean          | 445      |
|    gen/rollout/ep_rew_wrapped_mean  | 256      |
|    gen/time/fps                     | 3048     |
|    gen/time/iterations              | 1        |
|    gen/time/time_elapsed            | 5        |
|    gen/time/total_timesteps         | 196608   |
|    gen/train/explained_variance     | 0.911    |
|    gen/train/is_line_search_success | 1        |
|    gen/train/kl_divergence_loss     | 0.00848  |
|    gen/train/learning_rate          | 0.001    |
|    gen/train/n_updates              | 11       |
|    gen/train/policy_objective       | 0.0219   |
|    gen/train/value_loss             | 6.01     |
--------------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc             

round: 100%|██████████| 12/12 [01:47<00:00,  8.96s/it]


In [ ]:
env_TRPO.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_TRPO, env_TRPO, 100, return_episode_rewards=True,
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 9.57 +/- 0.6819824044651006
Rewards after training: 495.8 +/- 13.60147050873544


Rewards before training: 9.57 +/- 0.6819824044651006

Rewards after training: 495.8 +/- 13.60147050873544

# BC-PPO

(imitation library, BC does not work as expected - RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)). Try run BC with cpu to avoid this issue (changing runtime type from GPU to CPU, and it can run on colab but having some issue on my Win10 laptop).

In [ ]:
import numpy as np
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

In [ ]:
SEED = 42
rng = np.random.default_rng(0)
env = make_vec_env(
    "CartPole-v1", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="CartPole-v1",
    venv=env,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-CartPole-v1.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
transitions = rollout.flatten_trajectories(rollouts)

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)
bc_trainer.train(n_epochs=2)
reward, _ = evaluate_policy(bc_trainer.policy, env, 10)
print("Reward:", reward)

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.693     |
|    neglogp        | 0.694     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


484batch [00:02, 199.52batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000281 |
|    entropy        | 0.281     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 89.3      |
|    loss           | 0.399     |
|    neglogp        | 0.399     |
|    prob_true_act  | 0.774     |
|    samples_so_far | 16032     |
---------------------------------


865batch [00:04, 199.62batch/s]
875batch [00:04, 177.86batch/s]


Reward: 470.4


In [ ]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

rng = np.random.default_rng(0)
env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=rng,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],  # for computing rollouts
)


def train_expert():
    # note: use `download_expert` instead to download a pretrained, competent expert
    print("Training a expert.")
    expert = PPO(
        policy=MlpPolicy,
        env=env,
        seed=0,
        batch_size=64,
        ent_coef=0.0,
        learning_rate=0.0003,
        n_epochs=10,
        n_steps=64,
    )
    expert.learn(1_000)  # Note: change this to 100_000 to train a decent expert.
    return expert


def download_expert():
    print("Downloading a pretrained expert.")
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals-CartPole-v0",
        venv=env,
    )
    return expert


def sample_expert_transitions():
    # expert = train_expert()  # uncomment to train your own expert
    expert = download_expert()

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=None, min_episodes=50),
        rng=rng,
    )
    return rollout.flatten_trajectories(rollouts)


transitions = sample_expert_transitions()
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

evaluation_env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=rng,
    env_make_kwargs={"render_mode": "human"},  # for rendering
)

print("Evaluating the untrained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=True,  # comment out to speed up
)
print(f"Reward before training: {reward}")

print("Training a policy using Behavior Cloning")
bc_trainer.train(n_epochs=1)

print("Evaluating the trained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=True,  # comment out to speed up
)
print(f"Reward after training: {reward}")

ppo-seals-CartPole-v0.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

Sampling expert transitions.
Evaluating the untrained policy.
Reward before training: 8.0
Training a policy using Behavior Cloning


0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.693     |
|    neglogp        | 0.694     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


496batch [00:02, 223.20batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000388 |
|    entropy        | 0.388     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 93.7      |
|    loss           | 0.267     |
|    neglogp        | 0.268     |
|    prob_true_act  | 0.799     |
|    samples_so_far | 16032     |
---------------------------------


865batch [00:04, 207.19batch/s]
875batch [00:04, 210.10batch/s]


Evaluating the trained policy.
Reward after training: 500.0


# BC-TRPO

In [ ]:
import numpy as np
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

In [ ]:
SEED = 42
rng = np.random.default_rng(0)
env_TRPO = make_vec_env(
    "CartPole-v1",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert_TRPO = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="CartPole-v1",
    venv=env_TRPO,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-CartPole-v1.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
rollouts_TRPO = rollout.rollout(
    expert_TRPO,
    env_TRPO,
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=np.random.default_rng(SEED), # 随机数？ The random state to use for sampling trajectories.
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
transitions = rollout.flatten_trajectories(rollouts)

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)
bc_trainer.train(n_epochs=2)
reward, _ = evaluate_policy(bc_trainer.policy, env, 10)
print("Reward:", reward)

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.693     |
|    neglogp        | 0.694     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


484batch [00:02, 199.52batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000281 |
|    entropy        | 0.281     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 89.3      |
|    loss           | 0.399     |
|    neglogp        | 0.399     |
|    prob_true_act  | 0.774     |
|    samples_so_far | 16032     |
---------------------------------


865batch [00:04, 199.62batch/s]
875batch [00:04, 177.86batch/s]


Reward: 470.4


# RecurrentPPO
not quite exchangable for PPO/TRPO

In [ ]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

model = RecurrentPPO("MlpLstmPolicy", "CartPole-v1", verbose=1)
model.learn(5000)

vec_env = model.get_env()
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=20, warn=False)
print(mean_reward)

model.save("ppo_recurrent")
del model # remove to demonstrate saving and loading

model = RecurrentPPO.load("ppo_recurrent")

obs = vec_env.reset()
# cell and hidden state of the LSTM
lstm_states = None
num_envs = 1
# Episode start signals are used to reset the lstm states
episode_starts = np.ones((num_envs,), dtype=bool)
while True:
    action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    episode_starts = dones
    vec_env.render("")

Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20       |
|    ep_rew_mean     | 20       |
| time/              |          |
|    fps             | 377      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 25.5         |
|    ep_rew_mean          | 25.5         |
| time/                   |              |
|    fps                  | 122          |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 8.374453e-06 |
|    clip_fraction        | 0         

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


# DI-Engine try

In [ ]:
!pip install DI-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.3/185.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.2/732.2 kB 51.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/10

In [ ]:
import gym

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from dizoo.classic_control.cartpole.config.cartpole_dqn_config import main_config, create_config
from ding.config import compile_config

cfg = compile_config(main_config, create_cfg=create_config, auto=True)

In [ ]:
from ding.envs import DingEnvWrapper, BaseEnvManagerV2

collector_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("CartPole-v0")) for _ in range(cfg.env.collector_env_num)],
    cfg=cfg.env.manager
)
evaluator_env = BaseEnvManagerV2(
    env_fn=[lambda: DingEnvWrapper(gym.make("CartPole-v0")) for _ in range(cfg.env.evaluator_env_num)],
    cfg=cfg.env.manager
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
from ding.model import DQN
from ding.policy import DQNPolicy
from ding.data import DequeBuffer

model = DQN(**cfg.policy.model)
buffer_ = DequeBuffer(size=cfg.policy.other.replay_buffer.replay_buffer_size)
policy = DQNPolicy(cfg.policy, model=model)

In [ ]:
from ding.framework import task
from ding.framework.context import OnlineRLContext
from ding.framework.middleware import OffPolicyLearner, StepCollector, interaction_evaluator, data_pusher, eps_greedy_handler, CkptSaver

import logging
logging.getLogger().setLevel(logging.INFO)

with task.start(async_mode=False, ctx=OnlineRLContext()):
    # Evaluating, we place it on the first place to get the score of the random model as a benchmark value
    task.use(interaction_evaluator(cfg, policy.eval_mode, evaluator_env))
    task.use(eps_greedy_handler(cfg))  # Decay probability of explore-exploit
    task.use(StepCollector(cfg, policy.collect_mode, collector_env))  # Collect environmental data
    task.use(data_pusher(cfg, buffer_))  # Push data to buffer
    task.use(OffPolicyLearner(cfg, policy.learn_mode, buffer_))  # Train the model
    task.use(CkptSaver(policy, cfg.exp_name, train_freq=100))  # Save the model
    # In the evaluation process, if the model is found to have exceeded the convergence value, it will end early here
    task.run()

INFO:root:Evaluation: Train Iter(0) Env Step(0) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(8) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(16) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(24) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(32) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(40) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(48) Episode Return(12.200) 
INFO:root:Evaluation: Train Iter(0) Env Step(56) Episode Return(12.200) 
INFO:root:Training: Train Iter(0)	Env Step(64)	Loss(0.984)
INFO:root:Evaluation: Train Iter(40) Env Step(120) Episode Return(9.400) 
INFO:root:Evaluation: Train Iter(80) Env Step(184) Episode Return(21.800) 
INFO:root:collect end:
episode_count: 10
envstep_count: 125
train_sample_count: 500
avg_envstep_per_episode: 12.5
avg_sample_per_episode: 50.0
avg_envstep_per_sec: 9982.61142752421
avg_train_sample_per_sec: 399

# DI-Engine GAIL
since it does not has expert data, we train a dqn as the expert, then perform gail

## preparations for the gail pipeline

In [ ]:
from typing import Optional, Callable, List, Any

from ding.policy import PolicyFactory
from ding.worker import IMetric, MetricSerialEvaluator


class AccMetric(IMetric):

    def eval(self, inputs: Any, label: Any) -> dict:
        return {'Acc': (inputs['logit'].sum(dim=1) == label).sum().item() / label.shape[0]}

    def reduce_mean(self, inputs: List[Any]) -> Any:
        s = 0
        for item in inputs:
            s += item['Acc']
        return {'Acc': s / len(inputs)}

    def gt(self, metric1: Any, metric2: Any) -> bool:
        if metric2 is None:
            return True
        if isinstance(metric2, dict):
            m2 = metric2['Acc']
        else:
            m2 = metric2
        return metric1['Acc'] > m2


def mark_not_expert(ori_data: List[dict]) -> List[dict]:
    for i in range(len(ori_data)):
        # Set is_expert flag (expert 1, agent 0)
        ori_data[i]['is_expert'] = 0
    return ori_data


def mark_warm_up(ori_data: List[dict]) -> List[dict]:
    # for td3_vae
    for i in range(len(ori_data)):
        ori_data[i]['warm_up'] = True
    return ori_data


def random_collect( #粘过来是为了用这个
        policy_cfg: 'EasyDict',  # noqa
        policy: 'Policy',  # noqa
        collector: 'ISerialCollector',  # noqa
        collector_env: 'BaseEnvManager',  # noqa
        commander: 'BaseSerialCommander',  # noqa
        replay_buffer: 'IBuffer',  # noqa
        postprocess_data_fn: Optional[Callable] = None
) -> None:  # noqa
    assert policy_cfg.random_collect_size > 0
    if policy_cfg.get('transition_with_policy_data', False):
        collector.reset_policy(policy.collect_mode)
    else:
        action_space = collector_env.action_space
        random_policy = PolicyFactory.get_random_policy(policy.collect_mode, action_space=action_space)
        collector.reset_policy(random_policy)
    collect_kwargs = commander.step()
    if policy_cfg.collect.collector.type == 'episode':
        new_data = collector.collect(n_episode=policy_cfg.random_collect_size, policy_kwargs=collect_kwargs)
    else:
        new_data = collector.collect(
            n_sample=policy_cfg.random_collect_size,
            random_collect=True,
            record_random_collect=False,
            policy_kwargs=collect_kwargs
        )  # 'record_random_collect=False' means random collect without output log
    if postprocess_data_fn is not None:
        new_data = postprocess_data_fn(new_data)
    replay_buffer.push(new_data, cur_collector_envstep=0)
    collector.reset_policy(policy.collect_mode)

In [ ]:
from typing import Optional, Tuple
import os
import torch
from ditk import logging
from functools import partial
from tensorboardX import SummaryWriter
from copy import deepcopy
import numpy as np

from ding.envs import get_vec_env_setting, create_env_manager
from ding.worker import BaseLearner, InteractionSerialEvaluator, BaseSerialCommander, create_buffer, \
    create_serial_collector
from ding.config import read_config, compile_config
from ding.policy import create_policy
from ding.reward_model import create_reward_model
from ding.utils import set_pkg_seed
from ding.entry import collect_demo_data
from ding.utils import save_file
#from .utils import random_collect


def save_reward_model(path, reward_model, weights_name='best'):
    path = os.path.join(path, 'reward_model', 'ckpt')
    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
    path = os.path.join(path, 'ckpt_{}.pth.tar'.format(weights_name))
    state_dict = reward_model.state_dict()
    save_file(path, state_dict)
    print('Saved reward model ckpt in {}'.format(path))


def serial_pipeline_gail(
        input_cfg: Tuple[dict, dict],
        expert_cfg: Tuple[dict, dict],
        seed: int = 0,
        model: Optional[torch.nn.Module] = None,
        max_train_iter: Optional[int] = int(1e10),
        max_env_step: Optional[int] = int(1e10),
        collect_data: bool = True,
) -> 'Policy':  # noqa
    """
    Overview:
        Serial pipeline entry for GAIL reward model.
    Arguments:
        - input_cfg (:obj:`Union[str, Tuple[dict, dict]]`): Config in dict type. \
            ``str`` type means config file path. \
            ``Tuple[dict, dict]`` type means [user_config, create_cfg].
        - expert_cfg (:obj:`Union[str, Tuple[dict, dict]]`): Expert config in dict type. \
            ``str`` type means config file path. \
            ``Tuple[dict, dict]`` type means [user_config, create_cfg].
        - seed (:obj:`int`): Random seed.
        - model (:obj:`Optional[torch.nn.Module]`): Instance of torch.nn.Module.
        - max_train_iter (:obj:`Optional[int]`): Maximum policy update iterations in training.
        - max_env_step (:obj:`Optional[int]`): Maximum collected environment interaction steps.
        - collect_data (:obj:`bool`): Collect expert data.
    Returns:
        - policy (:obj:`Policy`): Converged policy.
    """
    if isinstance(input_cfg, str):
        cfg, create_cfg = read_config(input_cfg)
    else:
        cfg, create_cfg = deepcopy(input_cfg)
    if isinstance(expert_cfg, str):
        expert_cfg, expert_create_cfg = read_config(expert_cfg)
    else:
        expert_cfg, expert_create_cfg = expert_cfg
    create_cfg.policy.type = create_cfg.policy.type + '_command'
    cfg = compile_config(cfg, seed=seed, auto=True, create_cfg=create_cfg, save_cfg=True)
    if 'data_path' not in cfg.reward_model:
        cfg.reward_model.data_path = cfg.exp_name
    # Load expert data
    if collect_data:
        if expert_cfg.policy.get('other', None) is not None and expert_cfg.policy.other.get('eps', None) is not None:
            expert_cfg.policy.other.eps.collect = -1
        if expert_cfg.policy.get('load_path', None) is None:
            expert_cfg.policy.load_path = cfg.reward_model.expert_model_path
        collect_demo_data(
            (expert_cfg, expert_create_cfg),
            seed,
            state_dict_path=expert_cfg.policy.load_path,
            expert_data_path=cfg.reward_model.data_path + '/expert_data.pkl',
            collect_count=cfg.reward_model.collect_count
        )
    # Create main components: env, policy
    env_fn, collector_env_cfg, evaluator_env_cfg = get_vec_env_setting(cfg.env)
    collector_env = create_env_manager(cfg.env.manager, [partial(env_fn, cfg=c) for c in collector_env_cfg])
    evaluator_env = create_env_manager(cfg.env.manager, [partial(env_fn, cfg=c) for c in evaluator_env_cfg])
    collector_env.seed(cfg.seed)
    evaluator_env.seed(cfg.seed, dynamic_seed=False)
    set_pkg_seed(cfg.seed, use_cuda=cfg.policy.cuda)
    policy = create_policy(cfg.policy, model=model, enable_field=['learn', 'collect', 'eval', 'command'])

    # Create worker components: learner, collector, evaluator, replay buffer, commander.
    tb_logger = SummaryWriter(os.path.join('./{}/log/'.format(cfg.exp_name), 'serial'))
    learner = BaseLearner(cfg.policy.learn.learner, policy.learn_mode, tb_logger, exp_name=cfg.exp_name)
    collector = create_serial_collector(
        cfg.policy.collect.collector,
        env=collector_env,
        policy=policy.collect_mode,
        tb_logger=tb_logger,
        exp_name=cfg.exp_name
    )
    evaluator = InteractionSerialEvaluator(
        cfg.policy.eval.evaluator, evaluator_env, policy.eval_mode, tb_logger, exp_name=cfg.exp_name
    )
    replay_buffer = create_buffer(cfg.policy.other.replay_buffer, tb_logger=tb_logger, exp_name=cfg.exp_name)
    commander = BaseSerialCommander(
        cfg.policy.other.commander, learner, collector, evaluator, replay_buffer, policy.command_mode
    )
    reward_model = create_reward_model(cfg.reward_model, policy.collect_mode.get_attribute('device'), tb_logger)

    # ==========
    # Main loop
    # ==========
    # Learner's before_run hook.
    learner.call_hook('before_run')

    # Accumulate plenty of data at the beginning of training.
    if cfg.policy.get('random_collect_size', 0) > 0:
        random_collect(cfg.policy, policy, collector, collector_env, commander, replay_buffer)
    best_reward = -np.inf
    while True:
        collect_kwargs = commander.step()
        # Evaluate policy performance
        if evaluator.should_eval(learner.train_iter):
            stop, reward = evaluator.eval(learner.save_checkpoint, learner.train_iter, collector.envstep)
            print("look here")
            print(reward)
            # reward_mean = np.array([r['eval_episode_return'] for r in reward]).mean()
            reward_mean = np.array(reward['eval_episode_return']).mean()
            if reward_mean >= best_reward:
                save_reward_model(cfg.exp_name, reward_model, 'best')
                best_reward = reward_mean
            if stop:
                break
        new_data_count, target_new_data_count = 0, cfg.reward_model.get('target_new_data_count', 1)
        while new_data_count < target_new_data_count:
            new_data = collector.collect(train_iter=learner.train_iter, policy_kwargs=collect_kwargs)
            new_data_count += len(new_data)
            # collect data for reward_model training
            reward_model.collect_data(new_data)
            replay_buffer.push(new_data, cur_collector_envstep=collector.envstep)
        # update reward_model
        reward_model.train()
        reward_model.clear_data()
        # Learn policy from collected data
        for i in range(cfg.policy.learn.update_per_collect):
            # Learner will train ``update_per_collect`` times in one iteration.
            train_data = replay_buffer.sample(learner.policy.get_attribute('batch_size'), learner.train_iter)
            if train_data is None:
                # It is possible that replay buffer's data count is too few to train ``update_per_collect`` times
                logging.warning(
                    "Replay buffer's data can only train for {} steps. ".format(i) +
                    "You can modify data collect config, e.g. increasing n_sample, n_episode."
                )
                break
            # update train_data reward using the augmented reward
            train_data_augmented = reward_model.estimate(train_data)
            learner.train(train_data_augmented, collector.envstep)
            if learner.policy.get_attribute('priority'):
                replay_buffer.update(learner.priority_info)
        if collector.envstep >= max_env_step or learner.train_iter >= max_train_iter:
            break

    # Learner's after_run hook.
    learner.call_hook('after_run')
    save_reward_model(cfg.exp_name, reward_model, 'last')
    # evaluate
    # evaluator.eval(learner.save_checkpoint, learner.train_iter, collector.envstep)
    return policy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# classic_control cartpole_dqn_gail_config.py
from easydict import EasyDict

cartpole_dqn_gail_config = dict(
    exp_name='cartpole_dqn_gail_seed0',
    env=dict(
        collector_env_num=8,
        evaluator_env_num=5,
        n_evaluator_episode=5,
        stop_value=195,
    ),
    reward_model=dict(
        type='gail',
        input_size=5,
        hidden_size=64,
        batch_size=64,
        learning_rate=1e-3,
        update_per_collect=100,
        # Users should add their own model path here. Model path should lead to a model.
        # Absolute path is recommended.
        # In DI-engine, it is ``exp_name/ckpt/ckpt_best.pth.tar``.
        # If collect_data is True, we will use this expert_model_path to collect expert data first, rather than we
        # will load data directly from user-defined data_path
        expert_model_path='cartpole_dqn_seed0/ckpt/ckpt_best.pth.tar',

        collect_count=1000,
    ),
    policy=dict(
        cuda=False,
        model=dict(
            obs_shape=4,
            action_shape=2,
            encoder_hidden_size_list=[128, 128, 64],
            dueling=True,
        ),
        nstep=1,
        discount_factor=0.97,
        learn=dict(
            batch_size=64,
            learning_rate=0.001,
            update_per_collect=3,
        ),
        collect=dict(n_sample=64),
        eval=dict(evaluator=dict(eval_freq=10, )),
        other=dict(
            eps=dict(
                type='exp',
                start=0.95,
                end=0.1,
                decay=10000,
            ),
            replay_buffer=dict(replay_buffer_size=20000, ),
        ),
    ),
)
cartpole_dqn_gail_config = EasyDict(cartpole_dqn_gail_config)
main_config = cartpole_dqn_gail_config
cartpole_dqn_gail_create_config = dict(
    env=dict(
        type='cartpole',
        import_names=['dizoo.classic_control.cartpole.envs.cartpole_env'],
    ),
    env_manager=dict(type='base'),
    policy=dict(type='dqn'),
)
cartpole_dqn_gail_create_config = EasyDict(cartpole_dqn_gail_create_config)
create_config = cartpole_dqn_gail_create_config


In [ ]:
# from ding.entry import serial_pipeline_gail
from dizoo.classic_control.cartpole.config import cartpole_dqn_config, cartpole_dqn_create_config
expert_main_config = cartpole_dqn_config
expert_create_config = cartpole_dqn_create_config
serial_pipeline_gail(
    (main_config, create_config), (expert_main_config, expert_create_config),
    max_env_step=1000000,
    seed=0,
    collect_data=True
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:268: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` i

Collect demo data successfully


INFO:evaluator_logger:[EVALUATOR]env 4 finish episode, final reward: 8.0000, current episode: 1
INFO:evaluator_logger:[EVALUATOR]env 2 finish episode, final reward: 9.0000, current episode: 2
INFO:evaluator_logger:[EVALUATOR]env 3 finish episode, final reward: 9.0000, current episode: 3
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 10.0000, current episode: 4
INFO:evaluator_logger:[EVALUATOR]env 0 finish episode, final reward: 11.0000, current episode: 5
INFO:evaluator_logger:
+-------+------------+---------------------+---------------+---------------+
| Name  | train_iter | ckpt_name           | episode_count | envstep_count |
+-------+------------+---------------------+---------------+---------------+
| Value | 0.000000   | iteration_0.pth.tar | 5.000000      | 55.000000     |
+-------+------------+---------------------+---------------+---------------+
+-------+-------------------------+---------------+---------------------+----------------------+
| Name  | avg

look here
{'eval_episode_return': [11.0, 10.0, 9.0, 9.0, 8.0], 'eval_episode_return_mean': 9.4}
Saved reward model ckpt in cartpole_dqn_gail_seed0_240228_171321/reward_model/ckpt/ckpt_best.pth.tar


INFO:buffer_logger:=== Sample data 0 Times ===
INFO:buffer_logger:
+-------+----------+----------+--------------+--------------+
| Name  | use_avg  | use_max  | priority_avg | priority_max |
+-------+----------+----------+--------------+--------------+
| Value | 1.000000 | 1.000000 | 1.000000     | 1.000000     |
+-------+----------+----------+--------------+--------------+
+-------+--------------+---------------+---------------+----------+
| Name  | priority_min | staleness_avg | staleness_max | beta     |
+-------+--------------+---------------+---------------+----------+
| Value | 1.000000     | 0.000000      | 0.000000      | 0.400006 |
+-------+--------------+---------------+---------------+----------+


INFO:learner_logger:[RANK0]: === Training Iteration 0 Result ===
INFO:learner_logger:
+-------+------------+----------------+-------------+--------------------+
| Name  | cur_lr_avg | total_loss_avg | q_value_avg | target_q_value_avg |
+-------+------------+----------------+------

...

INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 36.0000, current episode: 1
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 36.0000, current episode: 1
INFO:evaluator_logger:[EVALUATOR]env 4 finish episode, final reward: 96.0000, current episode: 2
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 36.0000, current episode: 2
INFO:evaluator_logger:[EVALUATOR]env 0 finish episode, final reward: 112.0000, current episode: 3
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 36.0000, current episode: 3
INFO:evaluator_logger:[EVALUATOR]env 3 finish episode, final reward: 166.0000, current episode: 4
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 36.0000, current episode: 4
INFO:evaluator_logger:[EVALUATOR]env 4 finish episode, final reward: 96.0000, current episode: 4
INFO:evaluator_logger:[EVALUATOR]env 2 finish episode, final reward: 200.0000, current episode: 5
INFO:evaluator_logger:
+---

look here
{'eval_episode_return': [112.0, 36.0, 200.0, 166.0, 96.0], 'eval_episode_return_mean': 122.0}


INFO:learner_logger:[RANK0]: === Training Iteration 1100 Result ===
INFO:learner_logger:
+-------+------------+----------------+-------------+--------------------+
| Name  | cur_lr_avg | total_loss_avg | q_value_avg | target_q_value_avg |
+-------+------------+----------------+-------------+--------------------+
| Value | 0.001000   | 6.945438       | 7.002361    | 5.840093           |
+-------+------------+----------------+-------------+--------------------+


INFO:evaluator_logger:[EVALUATOR]env 0 finish episode, final reward: 200.0000, current episode: 1
INFO:evaluator_logger:[EVALUATOR]env 1 finish episode, final reward: 200.0000, current episode: 2
INFO:evaluator_logger:[EVALUATOR]env 2 finish episode, final reward: 200.0000, current episode: 3
INFO:evaluator_logger:[EVALUATOR]env 3 finish episode, final reward: 200.0000, current episode: 4
INFO:evaluator_logger:[EVALUATOR]env 4 finish episode, final reward: 200.0000, current episode: 5
INFO:evaluator_logger:
+-------+------------

look here
{'eval_episode_return': [200.0, 200.0, 200.0, 200.0, 200.0], 'eval_episode_return_mean': 200.0}
Saved reward model ckpt in cartpole_dqn_gail_seed0_240228_171321/reward_model/ckpt/ckpt_best.pth.tar
Saved reward model ckpt in cartpole_dqn_gail_seed0_240228_171321/reward_model/ckpt/ckpt_last.pth.tar


DI-engine DRL Policy
DQN(
  (encoder): FCEncoder(
    (act): ReLU()
    (init): Linear(in_features=4, out_features=128, bias=True)
    (main): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): ReLU()
    )
  )
  (head): DuelingHead(
    (A): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=2, bias=True)
      )
    )
    (V): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
)

## box2d bipedalwalker

In [ ]:
# dizoo/box2d/bipedalwalker/config/bipedalwalker_gail_sac_config.py
from easydict import EasyDict

obs_shape = 24
act_shape = 4
bipedalwalker_sac_gail_default_config = dict(
    exp_name='bipedalwalker_sac_gail_seed0',
    env=dict(
        collector_env_num=8,
        evaluator_env_num=5,
        # (bool) Scale output action into legal range.
        act_scale=True,
        n_evaluator_episode=5,
        stop_value=300,
        rew_clip=True,
        # The path to save the game replay
        replay_path=None,
    ),
    reward_model=dict(
        type='gail',
        input_size=obs_shape + act_shape,
        hidden_size=64,
        batch_size=64,
        learning_rate=1e-3,
        update_per_collect=100,
        # Users should add their own model path here. Model path should lead to a model.
        # Absolute path is recommended.
        # In DI-engine, it is ``exp_name/ckpt/ckpt_best.pth.tar``.
        expert_model_path='model_path_placeholder',
        # Path where to store the reward model
        reward_model_path='data_path_placeholder+/reward_model/ckpt/ckpt_best.pth.tar',
        # Users should add their own data path here. Data path should lead to a file to store data or load the stored data.
        # Absolute path is recommended.
        # In DI-engine, it is usually located in ``exp_name`` directory
        data_path='data_path_placeholder',
        collect_count=100000,
    ),
    policy=dict(
        cuda=False,
        priority=False,
        random_collect_size=1000,
        model=dict(
            obs_shape=obs_shape,
            action_shape=act_shape,
            twin_critic=True,
            action_space='reparameterization',
            actor_head_hidden_size=128,
            critic_head_hidden_size=128,
        ),
        learn=dict(
            update_per_collect=1,
            batch_size=128,
            learning_rate_q=0.001,
            learning_rate_policy=0.001,
            learning_rate_alpha=0.0003,
            ignore_done=True,
            target_theta=0.005,
            discount_factor=0.99,
            auto_alpha=True,
            value_network=False,
        ),
        collect=dict(
            n_sample=128,
            unroll_len=1,
        ),
        other=dict(replay_buffer=dict(replay_buffer_size=100000, ), ),
    ),
)
bipedalwalker_sac_gail_default_config = EasyDict(bipedalwalker_sac_gail_default_config)
main_config = bipedalwalker_sac_gail_default_config

bipedalwalker_sac_gail_create_config = dict(
    env=dict(
        type='bipedalwalker',
        import_names=['dizoo.box2d.bipedalwalker.envs.bipedalwalker_env'],
    ),
    env_manager=dict(type='subprocess'),
    policy=dict(
        type='sac',
        import_names=['ding.policy.sac'],
    ),
    replay_buffer=dict(type='naive', ),
)
bipedalwalker_sac_gail_create_config = EasyDict(bipedalwalker_sac_gail_create_config)
create_config = bipedalwalker_sac_gail_create_config


In [ ]:
from ding.entry import serial_pipeline_gail
from dizoo.box2d.bipedalwalker.config import bipedalwalker_sac_config, bipedalwalker_sac_create_config
expert_main_config = bipedalwalker_sac_config
expert_create_config = bipedalwalker_sac_create_config
serial_pipeline_gail(
    [main_config, create_config], [expert_main_config, expert_create_config], seed=0, collect_data=True
)

DependencyNotInstalled: box2D is not installed, run `pip install gym[box2d]`

## Another task other than cartpole (mujoco 版本有问题，后面看看能不能修一下)

In [ ]:
# dizoo/mujoco/config/ant_gail_sac_config.py
from easydict import EasyDict

obs_shape = 111
act_shape = 8
ant_sac_gail_config = dict(
    exp_name='ant_sac_gail_seed0',
    env=dict(
        env_id='Ant-v3',
        norm_obs=dict(use_norm=False, ),
        norm_reward=dict(use_norm=False, ),
        collector_env_num=1,
        evaluator_env_num=8,
        n_evaluator_episode=8,
        stop_value=6000,
    ),
    reward_model=dict(
        input_size=obs_shape + act_shape,
        hidden_size=256,
        batch_size=64,
        learning_rate=1e-3,
        update_per_collect=100,
        # Users should add their own model path here. Model path should lead to a model.
        # Absolute path is recommended.
        # In DI-engine, it is ``exp_name/ckpt/ckpt_best.pth.tar``.
        expert_model_path='model_path_placeholder',
        # Path where to store the reward model
        reward_model_path='data_path_placeholder+/reward_model/ckpt/ckpt_best.pth.tar',
        # Users should add their own data path here. Data path should lead to a file to store data or load the stored data.
        # Absolute path is recommended.
        # In DI-engine, it is usually located in ``exp_name`` directory
        data_path='data_path_placeholder',
        collect_count=300000,
    ),
    policy=dict(
        cuda=True,
        random_collect_size=25000,
        model=dict(
            obs_shape=obs_shape,
            action_shape=act_shape,
            twin_critic=True,
            action_space='reparameterization',
            actor_head_hidden_size=256,
            critic_head_hidden_size=256,
        ),
        learn=dict(
            update_per_collect=1,
            batch_size=256,
            learning_rate_q=1e-3,
            learning_rate_policy=1e-3,
            learning_rate_alpha=3e-4,
            ignore_done=False,
            target_theta=0.005,
            discount_factor=0.99,
            alpha=0.2,
            reparameterization=True,
            auto_alpha=False,
        ),
        collect=dict(
            n_sample=64,
            unroll_len=1,
        ),
        command=dict(),
        eval=dict(),
        other=dict(replay_buffer=dict(replay_buffer_size=1000000, ), ),
    ),
)

ant_sac_gail_config = EasyDict(ant_sac_gail_config)
main_config = ant_sac_gail_config

ant_sac_gail_create_config = dict(
    env=dict(
        type='mujoco',
        import_names=['dizoo.mujoco.envs.mujoco_env'],
    ),
    env_manager=dict(type='subprocess'),
    policy=dict(
        type='sac',
        import_names=['ding.policy.sac'],
    ),
    replay_buffer=dict(type='naive', ),
    reward_model=dict(type='gail'),
)
ant_sac_gail_create_config = EasyDict(ant_sac_gail_create_config)
create_config = ant_sac_gail_create_config


In [ ]:
#!pip install glfw
#!pip install mujoco
!pip3 install -U 'mujoco-py<2.2,>=2.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.1 MB/s eta 0:00:00


In [ ]:
from ding.entry import serial_pipeline_gail
from dizoo.mujoco.config.ant_sac_config import ant_sac_config, ant_sac_create_config

expert_main_config = ant_sac_config
expert_create_config = ant_sac_create_config
serial_pipeline_gail(
    [main_config, create_config], [expert_main_config, expert_create_config],
    max_env_step=10000000,
    seed=0,
    collect_data=True
)

DependencyNotInstalled: No module named 'mujoco_py'. (HINT: you need to install mujoco_py, and also perform the setup instructions here: https://github.com/openai/mujoco-py/.)

# file ops

In [ ]:
!zip -r "cartpole_dqn_seed0.zip" '/content/cartpole_dqn_seed0'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  adding: content/cartpole_dqn_seed0/ (stored 0%)
  adding: content/cartpole_dqn_seed0/formatted_total_config.py (deflated 70%)
  adding: content/cartpole_dqn_seed0/total_config.py (deflated 71%)
  adding: content/cartpole_dqn_seed0/ckpt/ (stored 0%)
  adding: content/cartpole_dqn_seed0/ckpt/eval.pth.tar (deflated 18%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_100.pth.tar (deflated 21%)
  adding: content/cartpole_dqn_seed0/ckpt/.ipynb_checkpoints/ (stored 0%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_400.pth.tar (deflated 19%)
  adding: content/cartpole_dqn_seed0/ckpt/ckpt_best.pth.tar (deflated 18%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_700.pth.tar (deflated 18%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_200.pth.tar (deflated 20%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_800.pth.tar (deflated 18%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_0.pth.tar (deflated 11%)
  adding: content/cartpole_dqn_seed0/ckpt/iteration_500.pt

In [ ]:
!rm -rf /content/cartpole_dqn_gail_seed0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
